# Observed correlation function

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import Corrfunc
import astropy.io
import cosmolopy.distance
import cosmolopy.constants

ImportError: No module named Corrfunc

In [ ]:
survey = astropy.io.fits.open('../BAM/a100springfull.fits')
end = 10359

ra = np.deg2rad(survey[1].data['RAdeg_HI'][:end])
dec = np.deg2rad(survey[1].data['DECdeg_HI'][:end])
dist = survey[1].data['Dist'][:end]

In [ ]:
survey[1].data.dtype.names

In [ ]:
wp()